# Spearman's Correlation Matrix

In [ ]:
# Read in the data from the csv file
import pandas as pd
CO2Data = pd.read_csv("../data/Terminos_lagoon_TA_DIC_2023_RawData.csv")

In [ ]:
# Printe the column names
print(CO2Data.columns)

## Adequacy Test

In [ ]:
# Select the columns that we need for the analysis

CO2Data_FA_vars = CO2Data[['depth_m', 'do_mg_l', 'sal_psu', 'turbidity_fnu', 'temp_c', 'ta_micromol_kg', 'dic_micromol_kg', 'chlorophy_microg_l']]

# Select numeric columns for factor analysis
numeric_columns = CO2Data_FA_vars.select_dtypes(include=['float64', 'int64']).dropna(axis=1)
numeric_values = numeric_columns.values


In [ ]:
# Perform Bartlett's test of sphericity
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity

# Perform Bartlett's test of sphericity
chi_square_value, p_value = calculate_bartlett_sphericity(numeric_values)

# Print results
print(f"The data is {'not ' if p_value < 0.05 else ''}an identity matrix.")
print(f"p-value: {p_value:.4f}, chi-square value: {chi_square_value:.4f}")

In [ ]:
# The KMO test measures the suitability of data for factor analysis. It is a measure of sampling adequacy.
from factor_analyzer.factor_analyzer import calculate_kmo

# Perform the Kaiser-Meyer-Olkin test
_, kmo_model = calculate_kmo(numeric_values)

# Print results with interpretation
print(f"KMO Value: {kmo_model:.2f}")
if kmo_model > 0.8:
    print("Suitability: Excellent")
elif 0.7 <= kmo_model <= 0.79:
    print("Suitability: Acceptable")
elif 0.6 <= kmo_model <= 0.69:
    print("Suitability: Mediocre")
else:
    print("Suitability: Poor")

print("Interpretation: A KMO value above 0.6 is considered acceptable for factor analysis.")


## Choosing the Number of Factors

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

# Apply PCA to compute the eigenvalues of the correlation matrix
pca = PCA()
pca.fit(numeric_values)

# Extract eigenvalues representing the variance explained by each component
eigenvalues = pca.explained_variance_
print("Eigenvalues:", eigenvalues)

# Apply the Kaiser criterion: retain components with eigenvalues > 1
n_factors = np.sum(eigenvalues > 1)
print(f"Number of factors to retain (Kaiser criterion): {n_factors}")


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
plt.plot(range(1, len(eigenvalues) + 1), eigenvalues, marker='o', linestyle='-')
plt.axhline(y=1, color='red', linestyle='--', label='Kaiser Criterion')
plt.title('Scree Plot')
plt.xlabel('Factor Number')
plt.ylabel('Eigenvalue')
plt.legend()
plt.show()

In [ ]:
from sklearn.decomposition import FactorAnalysis
from sklearn.preprocessing import StandardScaler
from tabulate import tabulate  # For displaying tables

# Select numeric columns for factor analysis
numeric_columns = CO2Data_FA_vars.select_dtypes(include=['float64', 'int64']).dropna(axis=1)
X = numeric_columns.values

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Perform factor analysis
fa = FactorAnalysis(n_components=2,  rotation='varimax')  # Adjust n_components as needed
X_factors = fa.fit_transform(X_scaled)

# Display the factor loadings in a table (rounded to 2 decimal places)
factor_loadings = pd.DataFrame(
    fa.components_.T, 
    index=numeric_columns.columns, 
    columns=[f'Factor{i+1}' for i in range(fa.n_components)]
).round(2)  # Round to 2 decimal places

print(tabulate(factor_loadings, headers='keys', tablefmt='grid'))

In [ ]:
def plot_fa_biplot_final(df, vars_numericas, group_col=None, label_col=None,
                         n_factors=2, title='Factor Analysis - Terminos Lagoon',
                         show_labels=True, save_path=None, figsize=(10, 10)):
    import matplotlib.pyplot as plt
    import numpy as np
    from sklearn.decomposition import FactorAnalysis
    from sklearn.preprocessing import StandardScaler
    import matplotlib
    from matplotlib.patheffects import withStroke

    # Datos y FA
    X = df[vars_numericas].dropna()
    df_clean = df.loc[X.index]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    fa = FactorAnalysis(n_components=n_factors)
    X_factors = fa.fit_transform(X_scaled)
    loadings = fa.components_.T * np.sqrt(fa.noise_variance_)[:, np.newaxis]

    # Agrupaciones
    groups = df_clean[group_col].values if group_col else None
    labels = df_clean[label_col].values if label_col else ["" for _ in range(len(X_factors))]
    unique_groups = np.unique(groups) if groups is not None else None

    # Paleta viridis corregida
    colormap = matplotlib.colormaps.get_cmap('viridis').resampled(len(unique_groups)) if groups is not None else None

    # Figura
    fig, ax = plt.subplots(figsize=figsize)
    ax.axhline(0, color='lightgray', lw=1, zorder=1)
    ax.axvline(0, color='lightgray', lw=1, zorder=1)
    circle = plt.Circle((0, 0), 1, color='gray', fill=False, linestyle='dashed', linewidth=1)
    ax.add_artist(circle)

    # Flechas
    arrow_scale = 4
    for i, var in enumerate(vars_numericas):
        x = loadings[i, 0] * arrow_scale
        y = loadings[i, 1] * arrow_scale
        ax.arrow(0, 0, x, y,
                 color='black', alpha=0.9,
                 head_width=0.12, head_length=0.15,
                 linewidth=2.4, length_includes_head=True, zorder=3)
        ax.text(x * 1.2, y * 1.2, var, fontsize=13,
                ha='center', va='center', weight='bold',
                path_effects=[withStroke(linewidth=3, foreground='white')],
                zorder=4)

    # Puntos y etiquetas
    if groups is not None:
        for i, g in enumerate(unique_groups):
            idx = groups == g
            ax.scatter(X_factors[idx, 0], X_factors[idx, 1],
                       label=g, alpha=0.85, s=70,
                       edgecolor='white', linewidth=0.6,
                       color=colormap(i), zorder=2)
            if show_labels:
                for j in np.where(idx)[0]:
                    ax.text(X_factors[j, 0], X_factors[j, 1], labels[j],
                            fontsize=6.5, alpha=0.5)
    else:
        ax.scatter(X_factors[:, 0], X_factors[:, 1], alpha=0.7, s=50, zorder=2)

    # Estética
    ax.set_xlabel('Factor 1', fontsize=14, weight='bold', family='serif')
    ax.set_ylabel('Factor 2', fontsize=14, weight='bold', family='serif')
    ax.set_title(title, fontsize=16, weight='bold', family='serif')
    if groups is not None:
        ax.legend(title=group_col, fontsize=10, title_fontsize=11, loc='best')
    ax.tick_params(labelsize=11)
    ax.set_aspect('equal')

    # Límites automáticos con margen
    margin = 0.5
    xlims = X_factors[:, 0].min() - margin, X_factors[:, 0].max() + margin
    ylims = X_factors[:, 1].min() - margin, X_factors[:, 1].max() + margin
    ax.set_xlim(*xlims)
    ax.set_ylim(*ylims)

    plt.tight_layout()

    # Guardar si se solicita
    if save_path:
        if save_path.endswith('.pdf') or save_path.endswith('.svg'):
            plt.savefig(save_path, bbox_inches='tight')
        else:
            plt.savefig(save_path, dpi=600, bbox_inches='tight')

    plt.show()


In [ ]:
plot_fa_biplot_final(
    df=CO2Data,
    vars_numericas=CO2Data_FA_vars.select_dtypes(include=['float64', 'int64']).columns.tolist(),
    group_col='season',
    label_col='sample',
    n_factors=2,
    title='Factor Analysis - Terminos Lagoon'
)